# Chevron curve for qudit (0-1)

The $x$-axis is the time, or the amplitude. We'll choose amplitude (actually Martinis chose time I guess, not really sure). It's easier to think of rabi amplitude as a dimensionless number, than to stick to a fixed amplitude (with no plan of doing it) and reasoning using the complicated normalization system of IBM (dt = 2/9 nanoseconds and all of that).

The $y$-axis is the detuning. We'll not really know the real frequency $\omega_{01}=(E_{|1\rangle}-E_{|0\rangle})/\hbar$. We'll just have to add detuning $\delta\omega$ to the estimated frequency $\tilde{\omega}$. In the benefits of hindsint, this will tell you exactly what to expect from the population of the qubit in the event of an unknown detuning, i.e. what's the state vector in the Bloch sphere?

In [ ]:
import pulse01

pulse => amp dur freq (method: json)